# CCD Reduction part 2 - Photon Statistics

This notebook will guide you through calculations needed for the CCD characterization lab.

Look for items labeled "student exercise" where you're supposed to fill in the blanks.

## Noise from photon counts

How much noise would we expect the dark electrons to add?  

First, recall how propagation of error works: the dark frames have two contributions to the noise-per-pixel, read noise and photon counting (Poisson) noise:

$ \sigma_{dark}^2 = \sigma_{dark current}^2 + \sigma_{read noise}^2$

Then recall that dark current is a Poisson process, so $\sigma_{dark current} = \sqrt{\lambda_{dark current}}$.

For our 10s frames, we estimate that $\lambda_{dark current} = \mu_{dark} - \mu_{bias} = $ *student exercise [this number should be your estimate of the dark current from the CCD Reduction Exercise Part 1]*,

which suggests that the dark noise should be $\sigma_{dark current} = \sqrt{\lambda_{dark current}} =$ *student exercise [answer this by filling out the cell below]*

In [1]:
# student exercise
mean_dark_current = 13.815112984837924 # copy this number from the previous notebook
theoretical_dark_current_sigma = np.sqrt(mean_dark_current)
theoretical_dark_current_sigma

3.7168687069679933

We have direct measurements of the noise, though.  We have measured $\sigma_{dark} = $ `mean_dark_noise` = *student exercise 10.870688076255952*

and $\sigma_{read noise} = $ `mean_readnoise` = *student exercise 10.557842457849686*.  We can then solve for the dark current noise:

$\sigma_{dark current} = \left(\sigma^2_{dark} - \sigma^2_{read noise}\right)^{1/2}$

In [2]:
mean_dark_noise = 10.870688076255952 # copy this number from the previous notebook
mean_read_noise = 10.557842457849686 # copy this number from the previous notebook
dark_current_sigma = (mean_dark_noise**2 - mean_read_noise**2)**0.5
dark_current_sigma

2.5891739776383953

We can now see that the theoretical error from the observed dark current is higher than the measured value by:

In [3]:
theoretical_dark_current_sigma / dark_current_sigma

1.4355422768299937

This difference occurs because of the CCD's _gain_.

## Gain

The CCD's reading device (analog-to-digital converter, ADC) has some gain, which gives the number of ADUs per electron (analog-digital unit, or "arbitrary data unit").

$ G = \frac{n_{ADU}}{n_{e^-}}$

We can do some math to figure out what the gain is, then.

For our detector, we have that the measured signal $S_{ADU}$ is going to be higher than the number of electrons by $G$, i.e.:

$S_{ADU} = S_{electrons} G $

Since the number of electrons follows counting (Poisson) statistics, we know that the standard deviation in electrons is:

$\sigma_{electrons} = S_{electrons}^{1/2}$



which means that we expect the noise in ADUs to be

$\sigma_{ADU} = \sigma_{electrons} G$

### Proof

You can prove this by using the propagation of error formula, $\sigma_z^2 = (dz/dx)^2 \sigma_x^2$.

Since $S_{ADU} = G S_{e^-}$, that means $dS_{ADU}/d_{e^-}=G$. 

Therefore, the error in ADU is:
$$\sigma_{ADU}^2 = \frac{d S_{ADU}}{d S_{e^-}}^2 \sigma_{e^-}^2 = G^2 \sigma_{e^-}^2$$
or
$$\sigma_{ADU} = G \sigma_{e^-}$$


### Find the Gain in terms of quantities we measured

We can replace $\sigma_{electrons}$ in the above equation with the square root of the number of electrons.

We then obtain a relation between the signal in ADUs $S_{ADU}$, the noise $\sigma_{ADU}$, and the gain $G$:

$\sigma_{ADU} = \sigma_{electrons} G = S_{electrons}^{1/2} G = \left(G S_{ADU}\right)^{1/2} G = \left(S_{ADU}  G\right)^{1/2}$

We can solve for the gain, since we can measure both $S_{ADU}$ and $\sigma_{ADU}$ directly:

$$G = \frac{\sigma^2_{ADU}}{S_{ADU}}$$

We calculated above the dark current in ADUs and the standard deviation of the dark current in ADUs.  Put these together to compute the gain:

In [4]:
# student exercise
gain = dark_current_sigma**2 / mean_dark_current
gain

0.485252773092574

You should get a gain value of 0.485252773092574

That gain means that it takes approximately _two electrons_ to produce one ADU.

The gain is also reported in the FITS header, so we can check our work:

In [5]:
from astropy.io import fits
header = fits.getheader('bias_001.FIT')
header['EGAIN']

1.55

Our recovered gain value does not agree with the header value.  We'll investigate this further later - maybe you will obtain a better estimate of the gain in the lab!